In [1]:
import pandas as pd
from datetime import datetime, timedelta

In [2]:
trumptweets = pd.read_csv('./tweets/Trump_Tweets.csv')
trumptweets = trumptweets.dropna()

In [3]:
trumptweets.head(5)

,source,text,created_at,retweet_count,favorite_count,is_retweet,id_str
0,Twitter Media Studio,Thank you to @MarthaRaddatz and @TerryMoran fo...,10-27-2019 21:24:55,11176.0,41087,false,1.188567e+18
1,Twitter for iPhone,RT @StateDept: Last night the United States br...,10-27-2019 16:50:08,16384.0,0,true,1.188498e+18
2,Twitter for iPhone,RT @WhiteHouse: Thank you to the service membe...,10-27-2019 16:49:45,11357.0,0,true,1.188498e+18
3,Twitter for iPhone,https://t.co/7esnNSoa5D,10-27-2019 16:25:12,25546.0,108756,false,1.188492e+18
4,Twitter for iPhone,https://t.co/yJ0VKdNxHP,10-27-2019 14:31:33,22275.0,76549,false,1.188463e+18


In [4]:
trumptweets = trumptweets.drop(["source", "id_str"], axis=1)

In [5]:
trumptweets['created_at']= pd.to_datetime(trumptweets['created_at'], format="%m-%d-%Y %H:%M:%S") 

In [6]:
trumptweets.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 39096 entries, 0 to 39160
Data columns (total 5 columns):
text              39096 non-null object
created_at        39096 non-null datetime64[ns]
retweet_count     39096 non-null float64
favorite_count    39096 non-null object
is_retweet        39096 non-null object
dtypes: datetime64[ns](1), float64(1), object(3)
memory usage: 1.8+ MB


In [7]:
# trumptweets.head(10)

In [8]:
# Now we need to get the tweets matched up with the stock movements...

In [9]:
stocks = pd.read_csv('./stocks/spx_preprocessed.csv')
stocks['Date']= pd.to_datetime(stocks['Date']) 

In [10]:
# tweet_dates = trumptweets["created_at"].tolist()

In [11]:
# Current stock prices (price before the tweet)
# stocks["Date"].max()

In [12]:
trumptweets = trumptweets[trumptweets["created_at"].between(str(stocks["Date"].min() - timedelta(days=1)), str(stocks["Date"].max() + timedelta(days=1)))]

In [13]:
# trumptweets["created_at"].max()

In [14]:
# trumptweets["created_at"].max().hour
# print(trumptweets["created_at"].tolist()[-1].date(), stocks["Date"].tolist()[0].date())

In [15]:
trumptweets = trumptweets.sort_values(by="created_at")
trumptweets = trumptweets.reset_index(drop=True)
stocks = stocks.sort_values(by="Date")
stocks = stocks.reset_index(drop=True)

In [16]:
stocks.head(10)

,Unnamed: 0,Date,Open,Close,1 Day Open,1 Day Close,2 Day Open,2 Day Close,EOW Close,SOW Open
0,0,2010-01-04,1116.560059,1132.989990,1132.660034,1136.520020,1135.709961,1137.140015,1144.979980,1145.959961
1,1,2010-01-05,1132.660034,1136.520020,1135.709961,1137.140015,1136.270020,1141.689941,1144.979980,1145.959961
2,2,2010-01-06,1135.709961,1137.140015,1136.270020,1141.689941,1140.520020,1144.979980,1144.979980,1145.959961
3,3,2010-01-07,1136.270020,1141.689941,1140.520020,1144.979980,1145.959961,1146.979980,1144.979980,1145.959961
4,4,2010-01-08,1140.520020,1144.979980,1145.959961,1146.979980,1143.810059,1136.219971,1144.979980,1145.959961
5,5,2010-01-11,1145.959961,1146.979980,1143.810059,1136.219971,1137.310059,1145.680054,1136.030029,1136.030029
6,6,2010-01-12,1143.810059,1136.219971,1137.310059,1145.680054,1145.680054,1148.459961,1136.030029,1136.030029
7,7,2010-01-13,1137.310059,1145.680054,1145.680054,1148.459961,1147.719971,1136.030029,1136.030029,1136.030029
8,8,2010-01-14,1145.680054,1148.459961,1147.719971,1136.030029,1136.030029,1150.229980,1136.030029,1136.030029
9,9,2010-01-15,1147.719971,1136.030029,1136.030029,1150.229980,1147.949951,1138.040039,1136.030029,1136.030029


In [17]:
def get_closest_value(arr, target, round_dir):
    n = len(arr)
    left = 0
    right = n - 1
    mid = 0

    # edge case - last or above all
    if target >= arr[n - 1]:
        return arr[n - 1]
    # edge case - first or below all
    if target <= arr[0]:
        return arr[0]
    # BSearch solution: Time & Space: Log(N)

    while left < right:
        mid = (left + right) // 2  # find the mid
        if target < arr[mid]:
            right = mid
        elif target > arr[mid]:
            left = mid + 1
        else:
            return arr[mid]

    if target < arr[mid]:
        return find_closest(arr[mid - 1], arr[mid], target, round_dir)
    else:
        return find_closest(arr[mid], arr[mid + 1], target, round_dir)


# findClosest
# We find the closest by taking the difference
# between the target and both values. It assumes
# that val2 is greater than val1 and target lies
# between these two. 
def find_closest(val1, val2, target, round_dir):
    if val1 == target:
        return val1
    elif val2 == target:
        return val2
    return min(val1, val2) if round_dir.lower() == "down" else max(val1, val2)
#     return val2 if target - val1 >= val2 - target else val1

In [32]:
tweet_dates, stock_dates = trumptweets["created_at"].tolist(), stocks["Date"].tolist()
before_dates, before_metrics, after_dates, after_metrics = [], [], [], []
for tweet_date in tweet_dates:
    if tweet_date.hour < 9:
        # Use previous day's close as before price
        stock_date = get_closest_value(stock_dates, datetime.combine(tweet_date.date()-timedelta(days=1), datetime.min.time()), "down")
        before_dates.append(stock_date)
        before_metrics.append("close")
        # Use current day's open as after price
        after_stock_date = get_closest_value(stock_dates, datetime.combine(tweet_date.date(), datetime.min.time()), "up")
        after_dates.append(after_stock_date)
        after_metrics.append("open")
#         print("Before 9 - Tweet Date: {}, Before: {}, Metric: {}, After: {}, Metric: {}".format(tweet_date, 
#                                                                                      before_dates[-1], 
#                                                                                      before_metrics[-1], 
#                                                                                      after_dates[-1], 
#                                                                                      after_metrics[-1]))
    elif tweet_date.hour >= 17:
        # Use current day's close as before price
        stock_date = get_closest_value(stock_dates, datetime.combine(tweet_date.date(), datetime.min.time()), "down")
        before_dates.append(stock_date)
        before_metrics.append("close")
        # Use next day's open as after price
        after_stock_date = get_closest_value(stock_dates, datetime.combine(tweet_date.date()+timedelta(days=1), datetime.min.time()), "up")
        after_dates.append(after_stock_date)
        after_metrics.append("open")
#         print("After 17 - Tweet Date: {}, Before: {}, Metric: {}, After: {}, Metric: {}".format(tweet_date, 
#                                                                                      before_dates[-1], 
#                                                                                      before_metrics[-1], 
#                                                                                      after_dates[-1], 
#                                                                                      after_metrics[-1]))
    else:
        # Use current day's open as before price
        stock_date = get_closest_value(stock_dates, datetime.combine(tweet_date.date(), datetime.min.time()), "down")
        before_dates.append(stock_date)
        if stock_date.date() == tweet_date.date():
            before_metrics.append("open")
        else:
            before_metrics.append("close")
        # Use current day's close as after price
        after_stock_date = get_closest_value(stock_dates, datetime.combine(tweet_date.date(), datetime.min.time()), "up")
        after_dates.append(after_stock_date)
        if after_stock_date.date() == tweet_date.date():
            after_metrics.append("close")
        else:
            after_metrics.append("open")
#         print("Else - Tweet Date: {}, Before: {}, Metric: {}, After: {}, Metric: {}".format(tweet_date, 
#                                                                                      before_dates[-1], 
#                                                                                      before_metrics[-1], 
#                                                                                      after_dates[-1], 
#                                                                                      after_metrics[-1]))

In [33]:
# len(before_dates)
# len(trumptweets.index)

In [34]:
# # UGLY... - FUCK THIS ILL DO SOMETHING ELSE...
# # Need to fix this so it takes time into account - if its before trading hours, we need the day before...
# # tweet_dates, stock_dates = list(reversed(trumptweets["created_at"].tolist())), list(reversed(stocks["Date"].tolist()))
# tweet_dates, stock_dates = trumptweets["created_at"].tolist(), stocks["Date"].tolist()
# tweet_date_idx, stock_date_idx = len(tweet_dates)-1, len(stock_dates) - 1
# before_dates, before_metrics = [], []
# while tweet_date_idx >= 0:
#     stock_date_idx = len(stock_dates)-1
#     while stock_date_idx >= 0 and tweet_date_idx >= 0:
# #         if stock_date_idx < 100 or tweet_date_idx < 100:
# #             print(stock_dates[stock_date_idx], tweet_dates[tweet_date_idx], stock_dates[stock_date_idx] <= tweet_dates[tweet_date_idx])
#         if tweet_dates[tweet_date_idx].hour <= 9:
#             # If tweet date hour is between 9 and 5, use current day's open as price
#             # If tweet date hour is after 5, use current day's close as price
#             if stock_dates[stock_date_idx] <= tweet_dates[tweet_date_idx]:
#                 before_dates.append(stock_dates[stock_date_idx])
#                 if stock_dates[stock_date_idx].date() == tweet_dates[tweet_date_idx].date():
#                     before_metrics.append("close" if tweet_dates[tweet_date_idx].hour >= 17 else "open")
#                 else:
#                     before_metrics.append("close")
# #                 tweet_date_idx -=1
#                 # Hacky tings
#                 stock_date_idx = min(stock_date_idx + 100, len(stock_dates)-1)
#                 break
#             else:
#                 stock_date_idx -= 1
#         else:
#             # If tweet date hour is before 9, use previous day's close as price
#             if stock_dates[stock_date_idx] < tweet_dates[tweet_date_idx]:
#                 before_dates.append(stock_dates[stock_date_idx])
#                 before_metrics.append("close")
# #                 tweet_date_idx -=1
#                 # Hacky tings
#                 stock_date_idx = min(stock_date_idx + 100, len(stock_dates)-1)
#                 break
#             else:
#                 stock_date_idx -= 1
#     if stock_date_idx < 0:
#         before_dates.append(None)
#         before_metrics.append(None)
#     tweet_date_idx -=1
after_metrics

['open',
 'close',
 'close',
 'open',
 'open',
 'close',
 'open',
 'open',
 'close',
 'open',
 'open',
 'close',
 'open',
 'close',
 'close',
 'open',
 'open',
 'close',
 'open',
 'close',
 'close',
 'open',
 'open',
 'close',
 'open',
 'close',
 'open',
 'close',
 'open',
 'close',
 'close',
 'close',
 'close',
 'close',
 'close',
 'close',
 'close',
 'close',
 'close',
 'open',
 'open',
 'close',
 'close',
 'close',
 'close',
 'close',
 'close',
 'close',
 'close',
 'close',
 'open',
 'open',
 'close',
 'open',
 'close',
 'open',
 'open',
 'open',
 'close',
 'close',
 'close',
 'close',
 'close',
 'close',
 'close',
 'close',
 'close',
 'close',
 'open',
 'close',
 'close',
 'close',
 'close',
 'close',
 'close',
 'open',
 'open',
 'open',
 'close',
 'open',
 'open',
 'open',
 'close',
 'close',
 'close',
 'close',
 'close',
 'open',
 'open',
 'open',
 'open',
 'open',
 'open',
 'open',
 'close',
 'close',
 'open',
 'open',
 'open',
 'open',
 'close',
 'close',
 'close',
 'open',
 'c

In [35]:
trumptweets["before_date"] = pd.Series(before_dates)
trumptweets["before_metric"] = pd.Series(before_metrics)
trumptweets["after_date"] = pd.Series(after_dates)
trumptweets["after_metric"] = pd.Series(after_metrics)

In [36]:
trumptweets[trumptweets.isnull().any(axis=1)]
# trumptweets.head(100)
# trumptweets.head(10)
# stocks.head(10)
# trumptweets.head(10)
# tweet_dates[-10:]
# stock_dates[-10:]

,text,created_at,retweet_count,favorite_count,is_retweet,before_date,before_metric,after_date,after_metric


In [37]:
trumptweets.head(60)

,text,created_at,retweet_count,favorite_count,is_retweet,before_date,before_metric,after_date,after_metric
0,Celebrity Apprentice returns to NBC Sunday 3/1...,2010-01-12 18:05:08,20.0,3,false,2010-01-12,close,2010-01-13,open
1,Trump Tycoon App for iPhone & iPod Touch - It'...,2010-01-15 16:28:02,23.0,20,false,2010-01-15,open,2010-01-15,close
2,"from Donald Trump: ""I saw Lady Gaga last night...",2010-01-21 16:58:43,3813.0,3223,false,2010-01-21,open,2010-01-21,close
3,Golf Channel & Donald Trump's World of Golf ho...,2010-01-22 20:51:00,12.0,1,false,2010-01-22,close,2010-01-25,open
4,Superbowl Sunday is a great American tradition...,2010-02-07 16:54:42,16.0,3,false,2010-02-05,close,2010-02-08,open
5,Donald Trump appearing today on CNN Internatio...,2010-02-10 15:17:56,7.0,1,false,2010-02-10,open,2010-02-10,close
6,Donald Trump appeared on the final episode of ...,2010-02-11 19:57:36,16.0,3,false,2010-02-11,close,2010-02-12,open
7,From Donald Trump: “I’m so proud of my wife Me...,2010-02-26 21:18:26,25.0,21,false,2010-02-26,close,2010-03-01,open
8,From Donald Trump: Andrea Bocelli @ Mar-a-Lago...,2010-03-05 14:51:32,7.0,6,false,2010-03-05,open,2010-03-05,close
9,The Celebrity Apprentice has a two-hour premie...,2010-03-12 20:30:52,23.0,7,false,2010-03-12,close,2010-03-15,open


In [38]:
# NEXT STEPS:
# - ACTUALLY GET THE PRICES FOR THESE DAYS TO GET PROPER INPUT/OUTPUT
# - SPLIT DATA INTO TRAIN/TEST SETS (MAKE SURE TO RANDOMIZE DATES!)
# - EDA (CLASS BALANCE, MOST COMMON WORDS, CLUSTERING FOR COMMON TOPICS)
# - CLASSIFICATION (RANDOM FOREST? SOFT MARGIN SVM?)

In [24]:
# trumptweets["created_at"].min()

Timestamp('2010-01-12 18:05:08')

In [25]:
# stocks["Date"].min()

Timestamp('2010-01-04 00:00:00')

In [26]:
# trumptweets = trumptweets[trumptweets["created_at"] > str(stocks["Date"].min() - timedelta(days=1))]

In [27]:
# # Same deal here...
# tweet_dates, stock_dates = list(reversed(trumptweets["created_at"].tolist())), stocks["Date"].tolist()
# tweet_date_idx, stock_date_idx = 0, 0
# after_dates = []
# while tweet_date_idx < len(tweet_dates):
#     while stock_date_idx < len(stock_dates) and tweet_date_idx < len(tweet_dates):
#         if (stock_dates[stock_date_idx]+timedelta(hours=8)) or >= tweet_dates[tweet_date_idx]:
#             after_dates.append(stock_dates[stock_date_idx])
#             tweet_date_idx += 1
#         else:
#             stock_date_idx += 1
#     if stock_date_idx >= len(stock_dates):
#         after_dates.append(None)
#     tweet_date_idx += 1

SyntaxError: invalid syntax (<ipython-input-27-cd916971f4c8>, line 7)

In [ ]:
# trumptweets["after_date"] = pd.Series(list(reversed(after_dates)))

In [ ]:
# trumptweets.head(60)